# APIs: requêtes HTTP

## Imports

In [9]:
import json
import requests

## Utiliser [Nominatim](https://nominatim.org/) pour connaître les coordonnées géographiques d'une adresse

In [11]:
address = "Avenue Franklin Roosevelt, Ixelles"

"""Retrieve coordinates from Open Street Map"""
headers = {'Referer': 'https://nominatim.openstreetmap.org/ui/search.html'}
url = "https://nominatim.openstreetmap.org/search"
data = {'q': address, 'format': 'json'}
resp = requests.get(url, data, headers=headers)
try:
    json_list = json.loads(resp.text)
except json.JSONDecodeError: # response is not in JSON as expected, probably the request is being blocked
    print(resp.text)
for item in json_list:
    display_name = item['display_name']
    short_name = display_name.split(", ")[0]
    lat = item['lat']
    lon = item['lon']
    print(f"{short_name} ({lat} - {lon})")


ULB (50.8116064 - 4.3805110)


## Utiliser REST Countries pour récupérer des informations sur un pays

https://restcountries.com/

In [ ]:
country_name = "Belgium"

base_url = "https://restcountries.com/v3.1/"
name_url = base_url + "name/"
code_url = base_url + "alpha/"
resp = requests.get(name_url + country_name)
country = resp.json()[0]
try:
    languages = country['languages']
    print(f"Languages: {', '.join(languages.values())}")
    border_codes = country['borders']
    border_names = []
    for code in border_codes:
        resp = requests.get(code_url + code)
        border_country = resp.json()[0]
        border_name = border_country["name"]["common"]
        border_names.append(border_name)
    print(f"Borders: {', '.join(border_names)}")
except KeyError:
    print("Unknown country, please use English or native name")

Languages: German, French, Dutch
Borders: France, Germany, Luxembourg, Netherlands


## Pour en savoir plus

In [31]:
import requests
import pandas as pd


API_KEY = "8CIDfZPH8Mqcnz9lZZSjYITONKhwDQxV"  
QUERY = "Belgium"           
BEGIN_DATE = "19140101"     
END_DATE = "19181231"

URL = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

params = {
    "q": QUERY,
    "begin_date": BEGIN_DATE,
    "end_date": END_DATE,
    "sort": "oldest",  
    "api-key": API_KEY
}

response = requests.get(URL, params=params)

try:
    # Vérifier si la requête a réussi
    response.raise_for_status()
    data = response.json()
    articles = data['response']['docs']

    # Extraire les informations utiles
    results = []
    for article in articles:
        try:
            title = article.get("headline", {}).get("main")
            date = article.get("pub_date")
            url = article.get("web_url")
            results.append({
                "Titre": title,
                "Date": date,
                "URL": url
            })
        except KeyError:
            # Si une clé est manquante pour un article, on ignore cet article
            continue

    # Créer un tableau Pandas si on a des résultats
    if results:
        df = pd.DataFrame(results)
        display(df)
        print(f"Nombre d'articles trouvés : {len(df)}")
    else:
        print("Aucun article trouvé pour cette recherche.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erreur HTTP : {http_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Erreur lors de la requête : {req_err}")
except KeyError as key_err:
    print(f"Erreur dans le traitement des données : {key_err}")
except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")

,Titre,Date,URL
0,MAYOR BEGINS WORK WITH FEW CALLERS; Tells His ...,1914-01-03T05:00:00Z,https://www.nytimes.com/1914/01/03/archives/ma...
1,BEAUTY AND LUXURY MARK AUTO SALON; Foreign Car...,1914-01-03T05:00:00Z,https://www.nytimes.com/1914/01/03/archives/be...
2,HUERTA SENTIMENT GROWS IN CAPITAL; Federal Act...,1914-01-04T05:00:00Z,https://www.nytimes.com/1914/01/04/archives/hu...
3,AUTO SHOW DRAWS INTERESTED THRONGS; Large Atte...,1914-01-06T05:00:00Z,https://www.nytimes.com/1914/01/06/archives/au...
4,Would Help to Reduce the Present Cost of Living.,1914-01-06T05:00:00Z,https://www.nytimes.com/1914/01/06/archives/wo...
5,FINANCIAL MARKETS; Stocks Irregular -- Money E...,1914-01-08T05:00:00Z,https://www.nytimes.com/1914/01/08/archives/fi...
6,"COMPANY OWES $4,500,000.; Many Foreign Banks C...",1914-01-09T05:00:00Z,https://www.nytimes.com/1914/01/09/archives/co...
7,HONORS KROONLAND MEN.; Belgium Bestows Decorat...,1914-01-09T05:00:00Z,https://www.nytimes.com/1914/01/09/archives/ho...
8,"MISS HARRISON HAS A $10,000 'CELLO; Mrs. Almer...",1914-01-11T05:00:00Z,https://www.nytimes.com/1914/01/11/archives/mi...
9,MRS. JANE ROBERTS DEAD.; Aged Widow of First P...,1914-01-11T05:00:00Z,https://www.nytimes.com/1914/01/11/archives/mr...


Nombre d'articles trouvés : 10


- Les API Rest: https://towardsdatascience.com/introduction-to-rest-apis-90b5d9676004
- La librairie [`requests`](https://requests.readthedocs.io/en/latest/)
- Développer une API en Python : https://openclassrooms.com/en/courses/4525361-realisez-un-dashboard-avec-tableau/5774811-creez-une-api-avec-flask